<a id="TableOfContents"></a>
# TABLE OF CONTENTS:
<li><a href='#imports'>Imports</a></li>
<li><a href="#Q1">Question 1</a></li>
<li><a href="#Q2">Question 2</a></li>
<li><a href="#Q3">Question 3</a></li>
<li><a href="#Q4">Question 4</a></li>
<li><a href="#Q5">Question 5</a></li>
<li><a href="#Q6">Question 6</a></li>
<li><a href="#Q7">Question 7</a></li>
<li><a href="#Bonus1">Bonus-1</a></li>
<li><a href="#Bonus2">Bonus-2</a></li>

<a id='imports'></a>
# IMPORTS:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [1]:
# Basic sheiza
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Stats/Explore
from scipy import stats

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# .py file(s)
import acquire
import prepare
import explore
import evaluation

Using the titanic data, in your classification-exercises repository, create a notebook, decision_tree.ipynb where you will do the following:

In [2]:
# Acquire titanic dataframe
titanic = prepare.prep_titanic()
titanic.sample()

,survived,pclass,sex,age,fare,embark_town,alone,sex_female,sex_male,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
176,0,3,male,29.699118,25.4667,Southampton,0,0,1,0,0,1


In [3]:
# Identify y_cols
y_titanic = 'survived'

In [4]:
# Identify x_cols
x_titanic = titanic.columns.to_list()
x_titanic = [col for col in x_titanic if col not in ['survived', 'sex', 'embark_town']]
x_titanic

['pclass',
 'age',
 'fare',
 'alone',
 'sex_female',
 'sex_male',
 'embark_town_Cherbourg',
 'embark_town_Queenstown',
 'embark_town_Southampton']

In [5]:
# split data and stratify on survived
titanic_train, titanic_validate, titanic_test = prepare.split(titanic)

What column do you want to stratify onsurvived
train.shape:(498, 12)
validate.shape:(214, 12)
test.shape:(179, 12)


In [6]:
# Verify split shapes
titanic_train.shape, titanic_validate.shape, titanic_test.shape

((498, 12), (214, 12), (179, 12))

<a id='Q1'></a>
### 1. What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [7]:
# titanic models sets
titanic_models = titanic_train['survived']
titanic_models = pd.DataFrame(titanic_models)
titanic_models = titanic_models.rename(columns={'survived' : 'actual'})
titanic_models

,actual
474,0
370,1
573,1
110,0
167,0
...,...
735,0
163,0
770,0
196,0


In [8]:
# Baseline is '0' or 'DID NOT survive'
titanic.survived.value_counts()
titanic_models['baseline'] = 0

In [9]:
# baseline accuracy is 61.62%
evaluation.accuracy(titanic_models, 'actual')

actual: 100.00%

baseline: 61.65%

HIGHEST VALUE = baseline: 61.65%
LOWEST VALUE = baseline: 61.65%


<a id='Q2'></a>
### 2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [10]:
titanic_train[x_titanic]

,pclass,age,fare,alone,sex_female,sex_male,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
474,3,22.000000,9.8375,1,1,0,0,0,1
370,1,25.000000,55.4417,0,0,1,1,0,0
573,3,29.699118,7.7500,1,1,0,0,1,0
110,1,47.000000,52.0000,1,0,1,0,0,1
167,3,45.000000,27.9000,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...
735,3,28.500000,16.1000,1,0,1,0,0,1
163,3,17.000000,8.6625,1,0,1,0,0,1
770,3,24.000000,9.5000,1,0,1,0,0,1
196,3,29.699118,7.7500,1,0,1,0,1,0


In [12]:
# Create 1st model as decision tree
clf1 = DecisionTreeClassifier()
clf1.fit(titanic_train[x_titanic], titanic_train[y_titanic])
model1_dt = clf1.predict(titanic_train[x_titanic])
titanic_models['model1_dt'] = model1_dt

<a id='Q3'></a>
### 3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [22]:
clf1_score = clf1.score(titanic_train[x_titanic], titanic_train[y_titanic])
clf1_confusion_matrix = confusion_matrix(titanic_models.actual, model1_dt)
clf1_class_report = classification_report(titanic_models.actual, model1_dt)
print(f'clf1 score:\n{clf1_score}\n')
print(f'clf1 confusion matrix:\n{clf1_confusion_matrix}\n')
print(f'clf1 classification report:\n{clf1_class_report}')

clf1 score:
0.9839357429718876

clf1 confusion matrix:
[[306   1]
 [  7 184]]

clf1 classification report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       307
           1       0.99      0.96      0.98       191

    accuracy                           0.98       498
   macro avg       0.99      0.98      0.98       498
weighted avg       0.98      0.98      0.98       498



<a id='Q4'></a>
### 4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.
<li><a href='#TableOfContents'>Table of Contents</a></li>

<a id='Q5'></a>
### 5. Run through steps 2-4 using a different max_depth value.
<li><a href='#TableOfContents'>Table of Contents</a></li>

<a id='Q6'></a>
### 6. Which model performs better on your in-sample data?
<li><a href='#TableOfContents'>Table of Contents</a></li>

<a id='Q7'></a>
### 7. Which model performs best on your out-of-sample data, the validate set?
<li><a href='#TableOfContents'>Table of Contents</a></li>

<a id='Bonus1'></a>
### BONUS-1. Work through these same exercises using the Telco dataset.
<li><a href='#TableOfContents'>Table of Contents</a></li>

<a id='Bonus2'></a>
### BONUS-2. Experiment with this model on other datasets with a higher number of output classes.
<li><a href='#TableOfContents'>Table of Contents</a></li>